# Imports

In [24]:
import os
import torch
import librosa
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from typing import List, Tuple
os.environ['HF_TOKEN'] = open('token.txt', 'r').read().strip()

'cuda:0'

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device